In [ ]:
import openai
import json
import math
from openai.lib._parsing._completions import type_to_response_format_param
import pandas as pd

import functions.prompts as prompts

client = openai.Client()

In [ ]:
model = "gpt-4o" # "o1-2024-12-17"

def wrap(id, body):
    return {
        "custom_id": id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": body,
    }
    
def gen_body(text):
    return {
            "model": model,
            "messages":[{
                "role": "user",
                "content": f"Generate {text} paper that could be published in a top-5 economics journal."
            }],
            "max_completion_tokens": 64000 if model == "o1" else 16384
        }

In [ ]:
def batch():
    fields = ['a finance', 'a macroeconomics', 'a microeconimics', 'an applied microeconomics']
    reqs =  [
        [ wrap(f"{field.split(" ")[1][:3]}_{i}", gen_body(field)) for i in range(5) ]
        for field in fields
    ]
    return [ x for xs in reqs for x in xs ]

with open(f"dump/input/fake_papers-{model}.jsonl", 'a') as f:
    f.write(f"{"\n".join([json.dumps(i) for i in batch()])}\n")

In [ ]:
batch_input_file = client.files.create(
    file=open(f"dump/input/fake_papers-{model}.jsonl", "rb"),
    purpose="batch"
)
print(f"id: {batch_input_file.id}")

In [ ]:
batch_input_file_id = batch_input_file.id
batch_object = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": f"[AI PAPER]"
    }
)
print(f"id: {batch_object.id}")

In [ ]:
r = client.batches.retrieve(batch_object.id)

In [ ]:
with open("fake_papers-4o.jsonl", "w") as f:
    f.write(client.files.content(r.output_file_id).text)

In [ ]:
for line in client.files.content(r.output_file_id).text.split("\n")[:-1]:
    l = json.loads(line)
    with open(f'./fake-papers/4o-{l["custom_id"]}.txt', 'w') as f:
        f.write(l["response"]["body"]["choices"][0]["message"]["content"])